In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import os
import projcore as pc
from sklearn.impute import KNNImputer


sns.set(style="whitegrid")


DATA_DIR = os.path.join(os.getcwd(), 'data')

In [2]:
def load_data(data_dir=DATA_DIR):
    """Load all datasets."""
    battles_df = pd.read_csv(os.path.join(data_dir, 'BattlesStaging_01012021_WL_tagged.csv'))
    clash_royal_data = pd.read_csv(os.path.join(data_dir, 'clash_royal_data.csv'))
    card_master_list = pd.read_csv(os.path.join(data_dir, 'CardMasterListSeason18_12082020.csv'))
    wincons = pd.read_csv(os.path.join(data_dir, 'Wincons.csv'))
    
    mapping = {
        'battles_df': battles_df,
        'clash_royal_data': clash_royal_data,
        'card_master_list': card_master_list,
        'wincons': wincons
    }
    
    return mapping

In [3]:
# load the actual data
datasets_mapping = load_data()
battles_df = datasets_mapping['battles_df']
clash_royal_data = datasets_mapping['clash_royal_data']
card_master_list = datasets_mapping['card_master_list']
wincons = datasets_mapping['wincons']

# Notes
1. 'data\CardMasterListSeason18_12082020.csv' has mappings between card id and card name. we already have this information inside the 'data\BattlesStaging_01012021_WL_tagged.csv' so we don't need to add it when merging the datasets together.

2. when merging with the 'data\Wincons.csv' we should probably create a new column for each card to see if it's a win condition or not.

3. when it comes to merging with 'data\clash_royal_data.csv', we should consider the name of each card and add the information for winner.card.{1..8} + loser.card.{1..8} and maybe have a summarizing column for the winner and loser where we calculate the weighted average of all the card infos

In [4]:
def add_wincon_info(battles_df, wincons):
    """
    Add a new column for each card to indicate if it's a win condition.
    """
    wincon_card_ids = set(wincons['card_id'])
    
    for i in range(1, 9):
        battles_df.loc[:, f'winner.card{i}.is_wincon'] = battles_df[f'winner.card{i}.id'].isin(wincon_card_ids)
        battles_df.loc[:, f'loser.card{i}.is_wincon'] = battles_df[f'loser.card{i}.id'].isin(wincon_card_ids)
    
    return battles_df

def clean_and_convert_data(clash_royal_data):
    """
    Clean and convert data in clash_royal_data for numerical calculations.
    """
    clash_royal_data['Usage'] = clash_royal_data['Usage'].astype(str)
    clash_royal_data['increase_in_usage'] = clash_royal_data['increase_in_usage'].astype(str)
    clash_royal_data['Win'] = clash_royal_data['Win'].astype(str)
    clash_royal_data['increase_in_win'] = clash_royal_data['increase_in_win'].astype(str)
    clash_royal_data['CWR'] = clash_royal_data['CWR'].astype(str)
    
    clash_royal_data['Usage'] = clash_royal_data['Usage'].str.rstrip('%').astype(float) / 100.0
    clash_royal_data['increase_in_usage'] = clash_royal_data['increase_in_usage'].str.rstrip('%').astype(float) / 100.0
    clash_royal_data['Win'] = clash_royal_data['Win'].str.rstrip('%').astype(float) / 100.0
    clash_royal_data['increase_in_win'] = clash_royal_data['increase_in_win'].str.rstrip('%').astype(float) / 100.0
    clash_royal_data['CWR'] = clash_royal_data['CWR'].str.rstrip('%').astype(float) / 100.0
    
    return clash_royal_data

def add_card_stats(battles_df, clash_royal_data):
    """
    Add all card stats (e.g., Rating, Usage, Win, CWR) from clash_royal_data to battles_df.
    """
    clash_royal_data['unique_id'] = clash_royal_data['name'] + '_' + clash_royal_data['Rating'].astype(str)   
    card_name_to_unique_id = clash_royal_data.set_index('name')['unique_id'].to_dict()
    card_stats = clash_royal_data.set_index('unique_id').to_dict(orient='index')  
    card_id_to_name = card_master_list.set_index('team.card1.id')['team.card1.name'].to_dict()

    stats_columns = ['Rating', 'Usage', 'increase_in_usage', 'Win', 'increase_in_win', 'CWR']
    new_columns = {}
    
    for i in range(1, 9):
        card_col = f'winner.card{i}.id'
        for stat in stats_columns:
            new_col = f'winner.card{i}.{stat.lower()}'
            new_columns[new_col] = battles_df[card_col].map(lambda x: card_stats.get(card_name_to_unique_id.get(card_id_to_name.get(x, ''), ''), {}).get(stat, np.nan))
    
    for i in range(1, 9):
        card_col = f'loser.card{i}.id'
        for stat in stats_columns:
            new_col = f'loser.card{i}.{stat.lower()}'
            new_columns[new_col] = battles_df[card_col].map(lambda x: card_stats.get(card_name_to_unique_id.get(card_id_to_name.get(x, ''), ''), {}).get(stat, np.nan))
    
    battles_df = pd.concat([battles_df, pd.DataFrame(new_columns)], axis=1)
    return battles_df

def summarize_card_stats(battles_df):
    """
    Calculate weighted averages for card stats for winner and loser.
    """
    # should add weighted average for win rate calculatinos but will do that after initial testing passes
    stats_columns = ['rating', 'usage', 'increase_in_usage', 'win', 'increase_in_win', 'cwr']
    
    for stat in stats_columns:
        battles_df[f'winner.avg_{stat}'] = battles_df[[f'winner.card{i}.{stat}' for i in range(1, 9)]].mean(axis=1)
    
    for stat in stats_columns:
        battles_df[f'loser.avg_{stat}'] = battles_df[[f'loser.card{i}.{stat}' for i in range(1, 9)]].mean(axis=1)
    
    return battles_df

def handle_missing_values(df, strategy='auto', n_neighbors=5):
    """
    Handle missing values based on each column's characteristics.
    """
    if strategy == 'auto':
        for column in df.columns:
            if df[column].isnull().sum() > 0:
                if df[column].dtype in [np.float64, np.int64]:
                    # For numerical columns, use median if skewed, otherwise use mean
                    if df[column].skew() > 1 or df[column].skew() < -1:
                        df[column] = df[column].fillna(df[column].median())
                    else:
                        df[column] = df[column].fillna(df[column].mean())
                else:
                    # For categorical columns, use mode
                    df[column] = df[column].fillna(df[column].mode()[0])
    elif strategy == 'knn':
        imputer = KNNImputer(n_neighbors)
        df[:] = imputer.fit_transform(df)
    elif strategy == 'drop':
        df = df.dropna()
    elif strategy == 'fill_zero':
        df = df.fillna(0)
    elif strategy == 'fill_mean':
        for column in df.columns:
            if df[column].dtype in [np.float64, np.int64]:
                df[column] = df[column].fillna(df[column].mean())
    elif strategy == 'fill_median':
        for column in df.columns:
            if df[column].dtype in [np.float64, np.int64]:
                df[column] = df[column].fillna(df[column].median())
    elif strategy == 'fill_mode':
        for column in df.columns:
            df[column] = df[column].fillna(df[column].mode()[0])
    elif strategy == 'interpolate':
        df = df.interpolate()
    elif strategy == None:
        pass
    else:
        raise ValueError(f"Unknown strategy: {strategy}")
    return df

def process_data(strategy=None, output_file='unified_clash_royale_data2.csv'):
    """Main function to process data with modular augmentations."""
    # Load data if not already loaded
    if 'battles_df' not in datasets_mapping:
        datasets_mapping.update(load_data())
    
    battles_df = datasets_mapping['battles_df']
    # battles_df = battles_df.head(100000) # to remove once testing is done on a small scale
    clash_royal_data = datasets_mapping['clash_royal_data']
    clash_royal_data = clean_and_convert_data(clash_royal_data)
    wincons = datasets_mapping['wincons']
    card_master_list = datasets_mapping['card_master_list']

    
    battles_df = pc.feature_engineering(battles_df, card_master_list, wincons)
    
    battles_df = add_wincon_info(battles_df, wincons)
    
    battles_df = add_card_stats(battles_df, clash_royal_data)
    
    battles_df = summarize_card_stats(battles_df)
    
    # battles_df = handle_missing_values(battles_df) # handle missing values later (after looking at the data)
    
    # battles_df.to_csv(os.path.join(DATA_DIR, output_file), index=False)
    # print(f"Unified data saved to {os.path.join(DATA_DIR, output_file)}")
    
    return battles_df

In [5]:
df = process_data(strategy='knn', output_file='unified_clash_royale_data_knn.csv')

In [6]:
def find_missing_values(df):
    """
    Find and display the count and percentage of missing values for each column in the DataFrame, sorted from highest to lowest percentage.
    """
    missing_values = df.isnull().sum()
    total_rows = df.shape[0]
    missing_percentage = (missing_values / total_rows) * 100
    
    missing_df = pd.DataFrame({
        'Missing Values': missing_values,
        'Percentage': missing_percentage
    })
    
    missing_df = missing_df[missing_df['Missing Values'] > 0].sort_values(by='Percentage', ascending=False)
    
    # print("Missing values in each column (sorted by percentage):")
    # for column in missing_df.index:
    #     print(f"{column}: {missing_df.loc[column, 'Missing Values']} missing values ({missing_df.loc[column, 'Percentage']:.2f}%)")
    
    return missing_df

def drop_rows_with_low_missing_rate(df, threshold=1.0):
    """
    Drop rows with missing values for columns where the missing rate is less than the given threshold.
    """
    missing_df = find_missing_values(df)
    columns_to_drop = missing_df[missing_df['Percentage'] < threshold].index
    
    print(f"\nDropping rows with missing values for columns with less than {threshold}% missing rate:")
    for column in columns_to_drop:
        print(f"Dropping rows for column: {column}")
        df = df.dropna(subset=[column])
    
    print(f"\nShape of DataFrame after dropping rows: {df.shape}")
    return df


In [7]:
print(f"The data frame has {df.shape[0]} rows and {df.shape[1]} columns.")
find_missing_values(df)

The data frame has 2823527 rows and 229 columns.


,Missing Values,Percentage
tournamentTag,2823527,100.000000
loser.princessTowersHitPoints,1462354,51.791748
princess_tower_gap,1462354,51.791748
loser.kingTowerHitPoints,784262,27.775970
loser.clan.tag,331196,11.729868
...,...,...
loser.card1.win,15677,0.555228
loser.card1.cwr,15677,0.555228
loser.card1.increase_in_win,15677,0.555228
loser_spell_troop_ratio,2297,0.081352


In [8]:
df = drop_rows_with_low_missing_rate(df, threshold=1.0)
print(f"The data frame has {df.shape[0]} rows and {df.shape[1]} columns.")
find_missing_values(df)


Dropping rows with missing values for columns with less than 1.0% missing rate:
Dropping rows for column: loser.card2.cwr
Dropping rows for column: loser.card2.usage
Dropping rows for column: loser.card2.increase_in_usage
Dropping rows for column: loser.card2.increase_in_win
Dropping rows for column: loser.card2.win
Dropping rows for column: loser.card2.rating
Dropping rows for column: winner.card2.rating
Dropping rows for column: winner.card2.usage
Dropping rows for column: winner.card2.increase_in_usage
Dropping rows for column: winner.card2.win
Dropping rows for column: winner.card2.increase_in_win
Dropping rows for column: winner.card2.cwr
Dropping rows for column: loser.card8.rating
Dropping rows for column: loser.card8.usage
Dropping rows for column: loser.card8.increase_in_usage
Dropping rows for column: loser.card8.increase_in_win
Dropping rows for column: loser.card8.win
Dropping rows for column: loser.card8.cwr
Dropping rows for column: winner.card7.increase_in_usage
Droppin

,Missing Values,Percentage
tournamentTag,2562651,100.000000
loser.princessTowersHitPoints,1318497,51.450510
princess_tower_gap,1318497,51.450510
loser.kingTowerHitPoints,706600,27.573009
loser.clan.tag,300409,11.722587
winner.clan.tag,271244,10.584508
win_lose_ratio,198123,7.731174
total_games,198123,7.731174
total_rank,198123,7.731174
loser.clan.badgeId,136515,5.327101


### Note:

1. removing data with nans in column where the missing percentage was less than 1% resulted in a total reduction of arond 10% of the data's volume. we should compare performance with this drop and without it.

2. "tournamentTag" is missing for all of the data so I think we should drop this column

In [9]:
df = df.drop(columns=["tournamentTag"])

In [ ]:
output_file='unified_clash_royale_data.csv'
df = handle_missing_values(df) # handle missing values later (after looking at the data)
    
df.to_csv(os.path.join(DATA_DIR, output_file), index=False)
print(f"Unified data saved to {os.path.join(DATA_DIR, output_file)}")

Unified data saved to c:\Users\Saleh\Desktop\Clash-Royale-Causal-analysis\data\unified_clash_royale_data_knn.csv


In [11]:
print(f"The data frame has {df.shape[0]} rows and {df.shape[1]} columns.")
find_missing_values(df)

The data frame has 2562651 rows and 228 columns.


,Missing Values,Percentage
